In [2]:
from data.create_dataloaders import create_dataloaders
from model.smooth_cross_entropy import smooth_crossentropy
from model.wide_res_net import WideResNet
import torch

from train import OptimizerType, train_multiple_models

batch_size = 128
threads = 2

depth = 16
width_factor = 8
dropout = 0.0

device = torch.device("cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
model = WideResNet(depth, width_factor, dropout, in_channels=3, labels=10).to(device)
train_dataloder, val_dataloader, test_dataloader = create_dataloaders(dataset_name="cifar10",
                                                                      batch_size=batch_size,
                                                                      num_workers=threads)

configs = [
    {"model": model, "criterion": smooth_crossentropy, "optimizer": {"optimizer_type": OptimizerType.SGD}, "num_epochs": 2, "model_name": "WRN-SGD"},
    {"model": model, "criterion": smooth_crossentropy, "optimizer": {"optimizer_type": OptimizerType.SAM}, "num_epochs": 2, "model_name": "WRN-SAM"},
    {"model": model, "criterion": smooth_crossentropy, "optimizer": {"optimizer_type": OptimizerType.ADAM}, "num_epochs": 2, "model_name": "WRN-ADAM"},
]

train_multiple_models(configs, train_dataloder, val_dataloader, test_dataloader, "cifar10", device=device)


Training model: WRN-SGD

Epoch 1/2


Training:   0%|          | 0/352 [00:00<?, ?it/s]/Users/jurahostic/anaconda3/envs/seminar_2/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Training:   0%|          | 0/352 [00:04<?, ?it/s]


RuntimeError: grad can be implicitly created only for scalar outputs